In [1]:
# Cellule 1 : Installation des packages
print("Installation des packages nécessaires...")

# Liste des packages à installer
packages = [
    "earthengine-api",  # Pour Google Earth Engine
    "pandas",           # Pour les données tabulaires
    "numpy",            # Pour les calculs numériques
    "matplotlib",       # Pour les graphiques
    "geemap"            # Facultatif : pour les cartes interactives
]

# Installer chaque package
import subprocess
import sys

for package in packages:
    try:
        # Vérifier si déjà installé
        __import__(package.replace("-", "_"))
        print(f"✓ {package} est déjà installé")
    except ImportError:
        print(f"Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installé avec succès")

print("\nInstallation terminée !")


Installation des packages nécessaires...
Installation de earthengine-api...
✓ earthengine-api installé avec succès
✓ pandas est déjà installé
✓ numpy est déjà installé
✓ matplotlib est déjà installé
✓ geemap est déjà installé

Installation terminée !


In [3]:
# Cellule 2 : Import des bibliothèques
print("Importation des bibliothèques...")

# 1. Google Earth Engine
try:
    import ee
    print("✓ Google Earth Engine importé")
except Exception as e:
    print(f"✗ Erreur avec Google Earth Engine: {e}")

# 2. Analyse de données
try:
    import pandas as pd
    print("✓ pandas importé (pour les tableaux)")
except Exception as e:
    print(f"✗ Erreur avec pandas: {e}")

# 3. Calcul numérique
try:
    import numpy as np
    print("✓ numpy importé (pour les calculs)")
except Exception as e:
    print(f"✗ Erreur avec numpy: {e}")

# 4. Visualisation
try:
    import matplotlib.pyplot as plt
    print("✓ matplotlib importé (pour les graphiques)")
except Exception as e:
    print(f"✗ Erreur avec matplotlib: {e}")

print("\nToutes les bibliothèques sont importées !")

Importation des bibliothèques...
✓ Google Earth Engine importé
✓ pandas importé (pour les tableaux)
✓ numpy importé (pour les calculs)
✓ matplotlib importé (pour les graphiques)

Toutes les bibliothèques sont importées !


In [5]:
# Cellule 3 : Initialisation avec TON projet
print("INITIALISATION AVEC TON PROJET GEE")
print("=" * 60)

# Ton project ID exact
TON_PROJET = "userscheikhthioub501"  # Ton projet
print(f"Project ID utilisé : {TON_PROJET}")

print("\nÉtape 1 : Importation de Google Earth Engine...")
import ee

print("Étape 2 : Initialisation...")
try:
    # Initialiser avec TON projet
    ee.Initialize(project=TON_PROJET)
    print("✓ SUCCÈS ! Google Earth Engine est initialisé")
    print(f"✓ Projet utilisé : {TON_PROJET}")
    
except Exception as e:
    print(f"✗ ERREUR : {e}")
    print("\nEssayons sans spécifier de projet...")
    try:
        ee.Initialize()
        print("✓ SUCCÈS avec initialisation simple !")
    except Exception as e2:
        print(f"✗ Échec aussi : {e2}")

INITIALISATION AVEC TON PROJET GEE
Project ID utilisé : userscheikhthioub501

Étape 1 : Importation de Google Earth Engine...
Étape 2 : Initialisation...
✓ SUCCÈS ! Google Earth Engine est initialisé
✓ Projet utilisé : userscheikhthioub501


In [7]:
# Cellule 4 : CHARGEMENT DE TES DONNÉES GADM
print("CHARGEMENT DE TES DONNÉES GADM DEPUIS TON ASSET")
print("=" * 70)

import ee

# TON Asset ID exact
TON_ASSET_ID = "projects/earthengine-legacy/assets/GADM/gadm41_SEN_2"
print(f"Asset ID utilisé : {TON_ASSET_ID}")

try:
    # Charger TES données
    senegal_gadm = ee.FeatureCollection(TON_ASSET_ID)
    
    # Compter les départements
    nb_depts = senegal_gadm.size().getInfo()
    print(f"✅ SUCCÈS ! {nb_depts} départements chargés")
    
    # Afficher les propriétés du premier département
    premier = senegal_gadm.first()
    props = premier.getInfo()['properties']
    
    print(f"\nPropriétés disponibles (10 premières) :")
    for i, (cle, valeur) in enumerate(list(props.items())[:10]):
        print(f"  {i+1:2d}. {cle:20} : {str(valeur)[:30]}...")
    
    # Chercher le nom du département
    noms_possibles = ['NAME_2', 'name_2', 'NOM', 'nom', 'NAME', 'ADM2_NAME', 'adm2_name']
    nom_colonne = None
    
    for nom in noms_possibles:
        if nom in props:
            nom_colonne = nom
            break
    
    if nom_colonne:
        print(f"\n✅ Colonne pour les noms : '{nom_colonne}'")
        
        # Afficher 5 premiers départements
        print("\n5 premiers départements :")
        depts = senegal_gadm.toList(5).getInfo()
        
        for i, dept in enumerate(depts):
            nom = dept['properties'].get(nom_colonne, 'Nom inconnu')
            # Chercher aussi la région
            region_cols = ['NAME_1', 'name_1', 'ADM1_NAME', 'REGION']
            region = 'Région inconnue'
            for reg in region_cols:
                if reg in dept['properties']:
                    region = dept['properties'][reg]
                    break
            
            print(f"  {i+1}. {nom:25} → Région: {region}")
    else:
        print("\n⚠️ Colonne de noms non trouvée")
        print(f"  Propriétés disponibles : {list(props.keys())}")
    
    print(f"\n✅ TES DONNÉES GADM SONT PRÊTES POUR L'ANALYSE !")
    
except Exception as e:
    print(f"❌ ERREUR : {e}")
    print("\nProblèmes possibles :")
    print("1. Asset ID incorrect")
    print("2. Shapefile pas encore complètement uploadé")
    print("3. Problème de permissions")
    print("\nOn utilise FAO GAUL en attendant...")
    
    senegal_gadm = ee.FeatureCollection("FAO/GAUL/2015/level2") \
        .filter(ee.Filter.eq('ADM0_NAME', 'Senegal'))
    print(f"✓ {senegal_gadm.size().getInfo()} départements (FAO temporaire)")

CHARGEMENT DE TES DONNÉES GADM DEPUIS TON ASSET
Asset ID utilisé : projects/earthengine-legacy/assets/GADM/gadm41_SEN_2
❌ ERREUR : Collection.loadTable: Name "projects/earthengine-legacy/assets/GADM/gadm41_SEN_2" is invalid.  Legacy assets under "projects/earthengine-legacy/assets/**" must have the top-level folders "users" or "projects" (e.g., "projects/earthengine-legacy/assets/users/foo/bar") and public assets must start with "projects/earthengine-public/

Problèmes possibles :
1. Asset ID incorrect
2. Shapefile pas encore complètement uploadé
3. Problème de permissions

On utilise FAO GAUL en attendant...
✓ 43 départements (FAO temporaire)


In [15]:
"""
SCRIPT UNIFIE - Tous les départements avec masque SCL 4, 6, 7
Masque commun : végétation + eau + non classifié
Seuil fixe : NDBI > 0
"""

import ee
import pandas as pd
import time
from datetime import datetime

# ============================================
# 1. INITIALISATION
# ============================================

try:
    ee.Initialize()
    print(" Google Earth Engine initialisé")
except Exception as e:
    print(" Erreur d'initialisation :", e)
    raise

# Configuration
ASSET_GADM = "projects/userscheikhthioub501/assets/gadm41_SEN_2"
ANNEE = 2018

print(f"\n Période d'analyse : {ANNEE}-01-01 à {ANNEE}-12-31")
print(f" Seuil fixe : NDBI > 0")
print(f" Masque SCL unifié : Classes 4, 6, 7")
print("   • 4: Végétation")
print("   • 6: Eau")
print("   • 7: Non classifié (inclut potentiellement zones urbaines)")

# ============================================
# 2. FONCTION DE MASQUAGE UNIFIE
# ============================================

def masquer_uniforme_467(image):
    """
    Masque unifié pour TOUS les départements
    Classes SCL : 4 (végétation), 6 (eau), 7 (non classifié)
    """
    scl = image.select('SCL')
    
    # Masque : seulement 4, 6, 7
    masque_valide = scl.eq(4).Or(scl.eq(6)).Or(scl.eq(7))
    
    # Appliquer le masque
    image_masquee = image.updateMask(masque_valide)
    
    # Normaliser les réflectances (toujours diviser par 10000)
    bands = ['B2', 'B3', 'B4', 'B8', 'B11']
    return image_masquee.select(bands).divide(10000)

# ============================================
# 3. CHARGEMENT ET PRÉPARATION
# ============================================

print("\n🔍 Chargement des images Sentinel-2...")

# Collection pour toute l'année
collection_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(ee.FeatureCollection(ASSET_GADM)) \
    .filterDate(f'{ANNEE}-01-01', f'{ANNEE}-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))

nombre_images_total = collection_s2.size().getInfo()
print(f"   Nombre total d'images : {nombre_images_total}")

# ============================================
# 4. ANALYSE PAR DÉPARTEMENT
# ============================================

print("\n📊 Analyse de tous les départements avec masque 4,6,7...")

senegal = ee.FeatureCollection(ASSET_GADM)
features = senegal.toList(senegal.size()).getInfo()



print(f"   Nombre total de départements : {len(features)}")

resultats = []

for i, feat in enumerate(features):
    try:
        feature = ee.Feature(feat)
        geom = feature.geometry()
        props = feat['properties']
        
        nom_dept = props.get('NAME_2', f'Dept_{i}')
        nom_region = props.get('NAME_1', '')
        
        print(f"   [{i+1}/{len(features)}] {nom_dept} ({nom_region})...")
        
        # ====================================
        # ÉTAPE 1 : Création du composite départemental
        # ====================================
        
        collection_dept = collection_s2 \
            .filterBounds(geom) \
            .map(masquer_uniforme_467)
        
        n_images = collection_dept.size().getInfo()
        
        
        
        if n_images > 0:
            # ====================================
            # ÉTAPE 2 : Création du composite (MOYENNE)
            # ====================================
            composite = collection_dept.mean().clip(geom)
            
            # ====================================
            # ÉTAPE 3 : Calcul du NDBI
            # ====================================
            ndbi = composite.normalizedDifference(['B11', 'B8']).rename('NDBI')
            
            # Calcul du NDBI moyen (pour information)
            stats_ndbi = ndbi.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geom,
                scale=30,
                maxPixels=1e9,
                bestEffort=True
            ).getInfo()
            
            ndbi_moyen = stats_ndbi.get('NDBI', 0) or 0
            
            # ====================================
            # ÉTAPE 4 : Classification avec seuil NDBI > 0
            # ====================================
            seuil = 0
            zones_baties = ndbi.gt(seuil).rename('bati')
            
            # Calcul du pourcentage de zones bâties
            stats_bati = zones_baties.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geom,
                scale=30,
                maxPixels=1e9,
                bestEffort=True
            ).getInfo()
            
            pct_bati = (stats_bati.get('bati', 0) or 0) * 100
            
            # ====================================
            # ÉTAPE 5 : Diagnostic et validation
            # ====================================
            
            # Diagnostic basé sur le type de département
            diagnostic = "OK"
            
            # Départements urbains majeurs
            depts_urbains_majeurs = ['Dakar', 'Pikine', 'Guédiawaye', 'Rufisque', 'Thiès', 'Saint-Louis']
            if nom_dept in depts_urbains_majeurs:
                if pct_bati < 5:
                    diagnostic = "URBAIN_TROP_FAIBLE"
                elif pct_bati > 90:
                    diagnostic = "URBAIN_TROP_ELEVE"
            
            # Départements urbains secondaires
            depts_urbains_secondaires = ['Mbour', 'Mbacké', 'Diourbel', 'Kaolack', 'Ziguinchor']
            if nom_dept in depts_urbains_secondaires:
                if pct_bati < 2:
                    diagnostic = "URBAIN_SECONDAIRE_FAIBLE"
                elif pct_bati > 80:
                    diagnostic = "URBAIN_SECONDAIRE_ELEVE"
            
            # Diagnostic général
            if pct_bati < 0.1:
                diagnostic = "TRES_FAIBLE"
            elif pct_bati > 95:
                diagnostic = "TRES_ELEVE"
            
            print(f"     → {pct_bati:.1f}% de zones bâties")
            print(f"       NDBI moyen: {ndbi_moyen:.3f}, Images: {n_images}")
            
            if diagnostic != "OK":
                print(f"        Diagnostic: {diagnostic}")
            
        else:
            pct_bati = -1
            ndbi_moyen = -999
            diagnostic = "PAS_D_IMAGES"
            print(f"      Aucune image disponible même après relaxation")
        
        # ====================================
        # ÉTAPE 6 : Enregistrement des résultats
        # ====================================
        
        resultat = {
            'CODE_REGION': props.get('GID_1', ''),
            'NOM_REGION': nom_region,
            'CODE_DEPT': props.get('GID_2', ''),
            'NOM_DEPT': nom_dept,
            'PCT_BATI': pct_bati,
            'NDBI_MOYEN': ndbi_moyen,
            'SEUIL_NDBI': 0,
            'IMAGES_DISPONIBLES': n_images,
            'DIAGNOSTIC': diagnostic,
            'STRATEGIE': 'UNIFORME_4_6_7',
            'PERIODE': f'{ANNEE}-01-01_{ANNEE}-12-31',
            'MASQUE_SCL': '4,6,7',
            'METHODE_COMPOSITE': 'MOYENNE',
            'DATE_TRAITEMENT': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        resultats.append(resultat)
        
        # Petite pause pour éviter les limites
        time.sleep(0.05)
        
    except Exception as e:
        nom_dept = props.get('NAME_2', f'Dept_{i}') if 'props' in locals() else f'Dept_{i}'
        error_msg = str(e)[:100]
        print(f" Erreur sur {nom_dept}: {error_msg}")
        
        resultats.append({
            'CODE_REGION': props.get('GID_1', '') if 'props' in locals() else '',
            'NOM_REGION': props.get('NAME_1', '') if 'props' in locals() else '',
            'CODE_DEPT': props.get('GID_2', '') if 'props' in locals() else '',
            'NOM_DEPT': nom_dept,
            'PCT_BATI': -1,
            'NDBI_MOYEN': -999,
            'SEUIL_NDBI': 0,
            'IMAGES_DISPONIBLES': 0,
            'DIAGNOSTIC': 'ERREUR',
            'STRATEGIE': 'UNIFORME_4_6_7',
            'PERIODE': f'{ANNEE}-01-01_{ANNEE}-12-31',
            'MASQUE_SCL': '4,6,7',
            'METHODE_COMPOSITE': 'MOYENNE',
            'DATE_TRAITEMENT': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
        continue

# ============================================
# 5. ANALYSE DES RÉSULTATS
# ============================================


print(" ANALYSE DES RÉSULTATS FINAUX")

df = pd.DataFrame(resultats)


if len(df) > 0:
    print(f"\n📊 DISTRIBUTION DES RÉSULTATS :")
    print(f"   % Bâti moyen : {df['PCT_BATI'].mean():.1f}%")
    print(f"   Médiane : {df['PCT_BATI'].median():.1f}%")
    print(f"   Minimum : {df['PCT_BATI'].min():.1f}%")
    print(f"   Maximum : {df['PCT_BATI'].max():.1f}%")
    print(f"   Écart-type : {df['PCT_BATI'].std():.1f}%")
    
   
  
    
    # Top 20 urbanisés
    print(f"\n TOP 20 DÉPARTEMENTS LES PLUS URBANISÉS :")
    top_20 = df.sort_values('PCT_BATI', ascending=False).head(20)
    for i, (_, row) in enumerate(top_20.iterrows(), 1):
        diag_indicator = "" if row['DIAGNOSTIC'] == 'OK' else f" ⚠️{row['DIAGNOSTIC']}"
        print(f"   {i:2}. {row['NOM_DEPT']:20} : {row['PCT_BATI']:6.1f}%{diag_indicator}")
    
    # Analyse par région
    print(f"\n ANALYSE PAR RÉGION :")
    regions_stats = df.groupby('NOM_REGION').agg({
        'PCT_BATI': ['mean', 'count', 'min', 'max'],
        'NOM_DEPT': 'first'
    }).round(1)
    
    regions_stats.columns = ['MOYENNE', 'NB_DEPTS', 'MIN', 'MAX', 'REGION']
    
    # Trier par % bâti moyen décroissant
    regions_stats = regions_stats.sort_values('MOYENNE', ascending=False)
    
    for region, data in regions_stats.iterrows():
        print(f"   {region:15} : {data['MOYENNE']:5.1f}% (min:{data['MIN']:4.1f}%, max:{data['MAX']:5.1f}%, {int(data['NB_DEPTS'])} depts)")

# ============================================
# 6. SAUVEGARDE ET EXPORT
# ============================================

print(f"\n Sauvegarde des résultats...")

# Nom du fichier avec timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
fichier_resultats = f"senegal_unifie_467_{timestamp}.csv"
df.to_csv(fichier_resultats, index=False, encoding='utf-8-sig')


print(f" Fichier complet sauvegardé : {fichier_resultats}")






 Google Earth Engine initialisé

 Période d'analyse : 2018-01-01 à 2018-12-31
 Seuil fixe : NDBI > 0
 Masque SCL unifié : Classes 4, 6, 7
   • 4: Végétation
   • 6: Eau
   • 7: Non classifié (inclut potentiellement zones urbaines)

🔍 Chargement des images Sentinel-2...
   Nombre total d'images : 375

📊 Analyse de tous les départements avec masque 4,6,7...
   Nombre total de départements : 45
   [1/45] Dagana (Saint-Louis)...
     → 33.4% de zones bâties
       NDBI moyen: -0.110, Images: 27
   [2/45] Podor (Saint-Louis)...
     → 35.1% de zones bâties
       NDBI moyen: -0.086, Images: 52
   [3/45] Saint-Louis (Saint-Louis)...
     → 76.6% de zones bâties
       NDBI moyen: 0.073, Images: 1
   [4/45] Bounkiling (Sédhiou)...
     → 46.9% de zones bâties
       NDBI moyen: -0.012, Images: 17
   [5/45] Goudomp (Sédhiou)...
     → 16.1% de zones bâties
       NDBI moyen: -0.091, Images: 29
   [6/45] Sédhiou (Sédhiou)...
     → 19.6% de zones bâties
       NDBI moyen: -0.058, Images: 29
   

In [26]:
"""
SCRIPT COMPLET - Visualisation Urbanisation Sénégal avec Cartes GADM
Auteur: Analyse Spatiale TP6
Date: Décembre 2024
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import geopandas as gpd
import unicodedata
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

# ============================================
# CHEMINS ET CONFIGURATION
# ============================================

REPERTOIRE_BASE = Path(r"C:\Users\HP\Documents\ISEP3\Semestre 1_CT\Stat\Stat_Spatiale\TP6")
CHEMIN_CSV = REPERTOIRE_BASE / "data" / "senegal_unifie_467_20251222_141659.csv"
# CORRECTION 1: Supprimer l'espace après "gadm" -> "gadm" pas "gadm "
CHEMIN_GADM = REPERTOIRE_BASE / "data" / "GADM" / "gadm" / "gadm41_SEN_shp"
DOSSIER_SORTIE = REPERTOIRE_BASE / "visualisations"

SHP_REGIONS = CHEMIN_GADM / "gadm41_SEN_1.shp"
SHP_DEPARTEMENTS = CHEMIN_GADM / "gadm41_SEN_2.shp"

# ============================================
# FONCTIONS UTILITAIRES
# ============================================

def normaliser_noms(nom):
    """Normalise les noms pour le matching"""
    nom = str(nom).upper().strip()
    nom = ''.join(c for c in unicodedata.normalize('NFD', nom) 
                  if unicodedata.category(c) != 'Mn')
    nom = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in nom)
    return ' '.join(nom.split())

def charger_donnees():
    """Charge toutes les données nécessaires"""
    print("="*70)
    print("CHARGEMENT DES DONNÉES")
    print("="*70)
    
    # CORRECTION 2: Ajouter un test pour vérifier que les fichiers existent
    print(f"📂 Vérification des fichiers...")
    print(f"  - CSV: {CHEMIN_CSV}")
    print(f"    Existe: {'✓' if CHEMIN_CSV.exists() else '✗'}")
    print(f"  - Shapefile régions: {SHP_REGIONS}")
    print(f"    Existe: {'✓' if SHP_REGIONS.exists() else '✗'}")
    print(f"  - Shapefile départements: {SHP_DEPARTEMENTS}")
    print(f"    Existe: {'✓' if SHP_DEPARTEMENTS.exists() else '✗'}")
    
    if not SHP_REGIONS.exists() or not SHP_DEPARTEMENTS.exists():
        print("\n❌ ERREUR: Fichiers shapefile non trouvés!")
        print(f"   Vérifiez que ces fichiers existent:")
        print(f"   {SHP_REGIONS}")
        print(f"   {SHP_DEPARTEMENTS}")
        print("\n   Essayez cette correction alternative:")
        print(f"   CHEMIN_GADM = REPERTOIRE_BASE / 'data' / 'GADM' / 'gadm41_SEN_shp'")
        return None, None, None, None
    
    # CSV
    print(f"\n📊 Chargement CSV...")
    df = pd.read_csv(CHEMIN_CSV, encoding='utf-8-sig')
    df = df[df['PCT_BATI'] >= 0].copy()
    print(f"✓ {len(df)} départements, {len(df)} valides")
    
    # Shapefiles
    print(f"\n🗺️  Chargement shapefiles GADM...")
    gdf_dept = gpd.read_file(SHP_DEPARTEMENTS)
    gdf_region = gpd.read_file(SHP_REGIONS)
    print(f"✓ {len(gdf_dept)} départements, {len(gdf_region)} régions")
    
    DOSSIER_SORTIE.mkdir(exist_ok=True)
    print(f"✓ Dossier sortie: {DOSSIER_SORTIE}")
    
    return df, df, gdf_dept, gdf_region

# ============================================
# GRAPHIQUES STATISTIQUES
# ============================================

def stats_descriptives(df):
    """Affiche statistiques"""
    print("\n" + "="*70)
    print("📊 STATISTIQUES DESCRIPTIVES")
    print("="*70)
    stats = df['PCT_BATI'].describe()
    print(f"\nMoyenne: {stats['mean']:.2f}% | Médiane: {stats['50%']:.2f}%")
    print(f"Min: {stats['min']:.2f}% | Max: {stats['max']:.2f}%")
    print(f"Écart-type: {stats['std']:.2f}%")

def graphique_distribution(df):
    """Distribution de l'urbanisation"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Distribution Urbanisation - Sénégal 2018', fontsize=16, fontweight='bold')
    
    # Histogramme
    axes[0,0].hist(df['PCT_BATI'], bins=30, color='steelblue', edgecolor='black')
    axes[0,0].axvline(df['PCT_BATI'].mean(), color='red', linestyle='--', 
                      label=f"Moy: {df['PCT_BATI'].mean():.1f}%")
    axes[0,0].set_title('Distribution des taux')
    axes[0,0].legend()
    axes[0,0].grid(alpha=0.3)
    
    # Boxplot
    axes[0,1].boxplot(df['PCT_BATI'], patch_artist=True)
    axes[0,1].set_title('Boxplot')
    axes[0,1].grid(alpha=0.3)
    
    # Cumulative
    sorted_data = np.sort(df['PCT_BATI'])
    cumulative = np.arange(1, len(sorted_data)+1) / len(sorted_data) * 100
    axes[1,0].plot(sorted_data, cumulative, 'b-', linewidth=2)
    axes[1,0].set_title('Distribution cumulative')
    axes[1,0].grid(alpha=0.3)
    
    # Catégories
    bins = [0, 1, 5, 10, 20, 100]
    labels = ['<1%', '1-5%', '5-10%', '10-20%', '>20%']
    cats = pd.cut(df['PCT_BATI'], bins=bins, labels=labels)
    counts = cats.value_counts().sort_index()
    colors = ['#2ecc71', '#f39c12', '#e67e22', '#e74c3c', '#c0392b']
    axes[1,1].bar(range(len(counts)), counts.values, color=colors, edgecolor='black')
    axes[1,1].set_xticks(range(len(counts)))
    axes[1,1].set_xticklabels(labels)
    axes[1,1].set_title('Catégories d\'urbanisation')
    
    plt.tight_layout()
    plt.savefig(DOSSIER_SORTIE / '01_distribution.png', dpi=300, bbox_inches='tight')
    print(f"✓ 01_distribution.png")
    plt.close()

def graphique_top_departements(df):
    """Top et Bottom départements"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10))
    fig.suptitle('Classement des Départements', fontsize=16, fontweight='bold')
    
    # Top 20
    top20 = df.nlargest(20, 'PCT_BATI')
    y = np.arange(len(top20))
    ax1.barh(y, top20['PCT_BATI'], color='steelblue', edgecolor='black')
    ax1.set_yticks(y)
    ax1.set_yticklabels([f"{r['NOM_DEPT']}" for _, r in top20.iterrows()], fontsize=9)
    ax1.set_title('Top 20 - Plus urbanisés', fontweight='bold')
    ax1.invert_yaxis()
    ax1.grid(alpha=0.3, axis='x')
    
    # Bottom 20
    bottom20 = df.nsmallest(20, 'PCT_BATI')
    y2 = np.arange(len(bottom20))
    ax2.barh(y2, bottom20['PCT_BATI'], color='coral', edgecolor='black')
    ax2.set_yticks(y2)
    ax2.set_yticklabels([f"{r['NOM_DEPT']}" for _, r in bottom20.iterrows()], fontsize=9)
    ax2.set_title('Bottom 20 - Moins urbanisés', fontweight='bold')
    ax2.invert_yaxis()
    ax2.grid(alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(DOSSIER_SORTIE / '02_top_departements.png', dpi=300, bbox_inches='tight')
    print(f"✓ 02_top_departements.png")
    plt.close()

def graphique_regions(df_valides):
    """Analyse par région"""
    stats_reg = df_valides.groupby('NOM_REGION')['PCT_BATI'].agg(['mean', 'count']).sort_values('mean', ascending=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10))  # Augmenté la largeur
    fig.suptitle('Analyse par Région - Sénégal 2018', fontsize=16, fontweight='bold')
    
    # Moyennes avec noms et valeurs
    y = np.arange(len(stats_reg))
    bars = ax1.barh(y, stats_reg['mean'], color='steelblue', edgecolor='black')
    ax1.set_yticks(y)
    
    # Noms des régions avec statistiques
    labels = []
    for idx, (region, row) in enumerate(stats_reg.iterrows()):
        label = f"{region}\n({row['count']} depts, {row['mean']:.1f}%)"
        labels.append(label)
    
    ax1.set_yticklabels(labels, fontsize=9)
    ax1.set_title('Taux moyen d\'urbanisation par région', fontweight='bold')
    ax1.invert_yaxis()
    ax1.grid(alpha=0.3, axis='x')
    ax1.set_xlabel('Taux d\'urbanisation (%)')
    
    # Ajouter les valeurs sur les barres
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax1.text(width + 0.2, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontsize=9)
    
    # Boxplots améliorés
    regions_order = stats_reg.index
    data = [df_valides[df_valides['NOM_REGION']==r]['PCT_BATI'].values for r in regions_order]
    bp = ax2.boxplot(data, labels=regions_order, patch_artist=True, showmeans=True)
    
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
    
    # Moyennes en rouge
    bp['means'][0].set_marker('D')
    bp['means'][0].set_markerfacecolor('red')
    bp['means'][0].set_markeredgecolor('red')
    
    ax2.set_xticklabels(regions_order, rotation=45, ha='right', fontsize=8)
    ax2.set_title('Distribution des taux par région', fontweight='bold')
    ax2.grid(alpha=0.3, axis='y')
    ax2.set_ylabel('Taux d\'urbanisation (%)')
    
    # Ajouter l'échelle et infos
    ax2.text(0.02, 0.98, f'Total: {len(df_valides)} départements',
            transform=ax2.transAxes, fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(DOSSIER_SORTIE / '03_analyse_regions.png', dpi=300, bbox_inches='tight')
    print(f"✓ 03_analyse_regions.png")
    plt.close()
    
    return stats_reg

# ============================================
# CARTES CHOROPLÈTHES
# ============================================
# ============================================
# CARTES CHOROPLÈTHES AMÉLIORÉES (sans mapclassify)
# ============================================

def carte_departements(df_valides, gdf_dept):
    """Carte départements avec noms et échelle"""
    # Normalisation et fusion
    df_valides['NOM_NORM'] = df_valides['NOM_DEPT'].apply(normaliser_noms)
    gdf_dept['NOM_NORM'] = gdf_dept['NAME_2'].apply(normaliser_noms)
    gdf = gdf_dept.merge(df_valides[['NOM_NORM', 'PCT_BATI', 'NDBI_MOYEN', 'NOM_REGION']], 
                         on='NOM_NORM', how='left')
    
    matched = gdf['PCT_BATI'].notna().sum()
    print(f"\n📍 Départements matchés: {matched}/{len(gdf)} ({matched/len(gdf)*100:.1f}%)")
    
    # Carte
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 12))
    fig.suptitle('ANALYSE SPATIALE DE L\'URBANISATION - SÉNÉGAL 2018\nCartographie par Département',
                 fontsize=18, fontweight='bold', y=0.98)
    
    # 1. TAUX D'URBANISATION
    # Classification manuelle (quantiles) au lieu de NaturalBreaks
    if gdf['PCT_BATI'].notna().sum() > 5:
        # Créer 5 classes basées sur les quantiles
        pct_data = gdf['PCT_BATI'].dropna()
        if len(pct_data) >= 5:
            quantiles = np.percentile(pct_data, [0, 20, 40, 60, 80, 100])
            # Ajuster les limites pour éviter les valeurs décimales trop précises
            quantiles = [round(q, 1) for q in quantiles]
            labels = [f'{quantiles[i]:.1f}-{quantiles[i+1]:.1f}%' for i in range(len(quantiles)-1)]
    
    # Carte choroplèthe avec classification
    gdf.plot(column='PCT_BATI', cmap='YlOrRd', linewidth=0.8, edgecolor='black', 
             legend=True, ax=ax1, missing_kwds={'color': 'lightgrey', 'label': 'Données manquantes'},
             legend_kwds={'label': 'Taux d\'Urbanisation (%)', 'shrink': 0.7, 
                         'orientation': 'horizontal', 'pad': 0.02})
    
    # Ajouter noms des départements principaux (top 15 pour éviter surcharge)
    top_depts = df_valides.nlargest(15, 'PCT_BATI')
    for _, dept in top_depts.iterrows():
        dept_norm = normaliser_noms(dept['NOM_DEPT'])
        dept_geom = gdf[gdf['NOM_NORM'] == dept_norm]
        if not dept_geom.empty and not dept_geom.geometry.is_empty.all():
            centroid = dept_geom.geometry.centroid.iloc[0]
            # Nom abrégé si trop long
            nom = dept['NOM_DEPT']
            if len(nom) > 15:
                nom = nom[:12] + '...'
            ax1.annotate(f"{nom}\n{dept['PCT_BATI']:.1f}%",
                        xy=(centroid.x, centroid.y), xytext=(3, 3),
                        textcoords="offset points", fontsize=7, fontweight='bold',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    ax1.set_title('TAUX D\'URBANISATION PAR DÉPARTEMENT', fontsize=14, fontweight='bold', pad=15)
    ax1.axis('off')
    
    # Échelle manuelle (sans matplotlib-scalebar)
    bounds = gdf.total_bounds
    # Calculer la distance en degrés pour 100 km (1 degré ≈ 111 km)
    scale_deg = 100 / 111  # Environ 0.9 degrés pour 100 km
    
    # Dessiner une barre d'échelle manuellement
    scale_x = bounds[0] + (bounds[2] - bounds[0]) * 0.05  # 5% depuis la gauche
    scale_y = bounds[1] + (bounds[3] - bounds[1]) * 0.05  # 5% depuis le bas
    
    ax1.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 
             color='black', linewidth=3)
    ax1.plot([scale_x, scale_x], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax1.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax1.text(scale_x + scale_deg/2, scale_y - 0.1, '100 km', 
             ha='center', va='top', fontsize=9, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Ajouter flèche nord
    north_x = bounds[0] + (bounds[2] - bounds[0]) * 0.95
    north_y = bounds[1] + (bounds[3] - bounds[1]) * 0.95
    ax1.text(north_x, north_y, 'N', fontsize=14, fontweight='bold',
             va='center', ha='center', 
             bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
    
    # 2. NDBI MOYEN
    gdf.plot(column='NDBI_MOYEN', cmap='RdYlGn_r', linewidth=0.8, edgecolor='black',
             legend=True, ax=ax2, missing_kwds={'color': 'lightgrey', 'label': 'Données manquantes'},
             legend_kwds={'label': 'NDBI Moyen (Built-up Index)', 'shrink': 0.7,
                         'orientation': 'horizontal', 'pad': 0.02})
    
    # Ajouter noms des régions (centroïdes régionaux)
    # Dissoudre par région pour obtenir les polygones régionaux
    try:
        gdf_regions = gdf.dissolve(by='NOM_REGION')
        for idx, region in gdf_regions.iterrows():
            if region.geometry.centroid.is_empty:
                continue
            centroid = region.geometry.centroid
            ax2.annotate(idx.upper(), xy=(centroid.x, centroid.y), fontsize=10,
                        fontweight='bold', ha='center', va='center',
                        bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    except:
        # Fallback: utiliser les noms des départements de chaque région
        regions = gdf['NOM_REGION'].dropna().unique()
        for region in regions:
            region_depts = gdf[gdf['NOM_REGION'] == region]
            if len(region_depts) > 0:
                centroid = region_depts.geometry.centroid.mean()
                ax2.annotate(region.upper(), xy=(centroid.x, centroid.y), fontsize=9,
                            fontweight='bold', ha='center', va='center',
                            bbox=dict(boxstyle='round,pad=0.4', facecolor='yellow', alpha=0.7))
    
    ax2.set_title('NDBI MOYEN PAR DÉPARTEMENT\n(Indice de Surface Bâtie Normalisé)', 
                 fontsize=14, fontweight='bold', pad=15)
    ax2.axis('off')
    
    # Échelle manuelle pour ax2
    ax2.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 
             color='black', linewidth=3)
    ax2.plot([scale_x, scale_x], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax2.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax2.text(scale_x + scale_deg/2, scale_y - 0.1, '100 km', 
             ha='center', va='top', fontsize=9, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Légende d'échelle commune
    fig.text(0.5, 0.02, 
             f'Source: Données GADM & Analyse Spatiale 2018 | Départements avec données: {matched}/{len(gdf)} | Projection: WGS84',
             ha='center', fontsize=9, style='italic')
    
    # Statistiques dans un encadré
    stats_text = f"""
    STATISTIQUES GLOBALES:
    • Taux moyen: {df_valides['PCT_BATI'].mean():.1f}%
    • Médiane: {df_valides['PCT_BATI'].median():.1f}%
    • Écart-type: {df_valides['PCT_BATI'].std():.1f}%
    • Min: {df_valides['PCT_BATI'].min():.1f}%
    • Max: {df_valides['PCT_BATI'].max():.1f}%
    • NDBI moyen: {df_valides['NDBI_MOYEN'].mean():.3f}
    • Départements: {len(df_valides)}
    """
    fig.text(0.02, 0.02, stats_text, fontsize=8,
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
             verticalalignment='bottom')
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(DOSSIER_SORTIE / '04_carte_departements.png', dpi=300, bbox_inches='tight')
    print(f"✓ 04_carte_departements.png")
    plt.close()


def carte_regions(df_valides, gdf_region):
    """Carte régions avec noms et échelle"""
    # Moyennes par région
    reg_moyennes = df_valides.groupby('NOM_REGION').agg({
        'PCT_BATI': ['mean', 'std', 'count'],
        'NDBI_MOYEN': 'mean'
    }).round(2)
    
    # Aplatir les colonnes
    reg_moyennes.columns = ['PCT_MEAN', 'PCT_STD', 'DEPT_COUNT', 'NDBI_MEAN']
    reg_moyennes = reg_moyennes.reset_index()
    reg_moyennes['NOM_NORM'] = reg_moyennes['NOM_REGION'].apply(normaliser_noms)
    
    # Fusion
    gdf_region['NOM_NORM'] = gdf_region['NAME_1'].apply(normaliser_noms)
    gdf = gdf_region.merge(reg_moyennes, on='NOM_NORM', how='left')
    
    matched = gdf['PCT_MEAN'].notna().sum()
    print(f"📍 Régions matchées: {matched}/{len(gdf)} ({matched/len(gdf)*100:.1f}%)")
    
    # Carte
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 12))
    fig.suptitle('ANALYSE RÉGIONALE DE L\'URBANISATION - SÉNÉGAL 2018\nCartographie par Région Administrative',
                 fontsize=18, fontweight='bold', y=0.98)
    
    # 1. TAUX MOYEN PAR RÉGION
    gdf.plot(column='PCT_MEAN', cmap='OrRd', linewidth=1.5, edgecolor='black',
             legend=True, ax=ax1, missing_kwds={'color': 'lightgrey', 'label': 'Données manquantes'},
             legend_kwds={'label': 'Taux d\'Urbanisation Moyen (%)', 'shrink': 0.7,
                         'orientation': 'horizontal', 'pad': 0.02})
    
    # Ajouter noms des régions avec valeurs
    for idx, row in gdf.iterrows():
        if pd.notna(row['PCT_MEAN']):
            centroid = row.geometry.centroid
            # Nom de la région en grand
            ax1.annotate(f"{row['NOM_REGION'].upper()}", 
                        xy=(centroid.x, centroid.y), xytext=(0, 12),
                        textcoords="offset points", fontsize=11, fontweight='bold',
                        ha='center', color='darkblue')
            # Valeur en dessous
            ax1.annotate(f"{row['PCT_MEAN']:.1f}% ± {row['PCT_STD']:.1f}%\n({row['DEPT_COUNT']} depts)",
                        xy=(centroid.x, centroid.y), xytext=(0, -12),
                        textcoords="offset points", fontsize=9,
                        ha='center', va='top',
                        bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.9))
    
    ax1.set_title('TAUX MOYEN D\'URBANISATION PAR RÉGION', fontsize=14, fontweight='bold', pad=15)
    ax1.axis('off')
    
    # Échelle manuelle
    bounds = gdf.total_bounds
    scale_x = bounds[0] + (bounds[2] - bounds[0]) * 0.05
    scale_y = bounds[1] + (bounds[3] - bounds[1]) * 0.05
    scale_deg = 100 / 111  # 100 km en degrés
    
    ax1.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 
             color='black', linewidth=3)
    ax1.plot([scale_x, scale_x], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax1.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax1.text(scale_x + scale_deg/2, scale_y - 0.1, '100 km', 
             ha='center', va='top', fontsize=9, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 2. NDBI MOYEN PAR RÉGION
    gdf.plot(column='NDBI_MEAN', cmap='RdYlGn', linewidth=1.5, edgecolor='black',
             legend=True, ax=ax2, missing_kwds={'color': 'lightgrey', 'label': 'Données manquantes'},
             legend_kwds={'label': 'NDBI Moyen par Région', 'shrink': 0.7,
                         'orientation': 'horizontal', 'pad': 0.02})
    
    # Ajouter valeurs NDBI
    for idx, row in gdf.iterrows():
        if pd.notna(row['NDBI_MEAN']):
            centroid = row.geometry.centroid
            ax2.annotate(f"NDBI: {row['NDBI_MEAN']:.3f}", 
                        xy=(centroid.x, centroid.y), fontsize=9, fontweight='bold',
                        ha='center', va='center',
                        bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.9))
    
    ax2.set_title('NDBI MOYEN PAR RÉGION\n(Indicateur de Surface Bâtie)', 
                 fontsize=14, fontweight='bold', pad=15)
    ax2.axis('off')
    
    # Échelle pour ax2
    ax2.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 
             color='black', linewidth=3)
    ax2.plot([scale_x, scale_x], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax2.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y-0.05, scale_y+0.05], 
             color='black', linewidth=2)
    ax2.text(scale_x + scale_deg/2, scale_y - 0.1, '100 km', 
             ha='center', va='top', fontsize=9, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Classement des régions
    reg_sorted = reg_moyennes.sort_values('PCT_MEAN', ascending=False)
    classement_text = "CLASSEMENT RÉGIONAL:\n"
    for i, (_, row) in enumerate(reg_sorted.iterrows(), 1):
        classement_text += f"{i}. {row['NOM_REGION']}: {row['PCT_MEAN']:.1f}%\n"
    
    fig.text(0.02, 0.25, classement_text, fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
             verticalalignment='top')
    
    # Statistiques régionales
    stats_text = f"""
    STATISTIQUES RÉGIONALES:
    • Régions: {len(reg_moyennes)}
    • Départements totaux: {reg_moyennes['DEPT_COUNT'].sum()}
    • Taux max: {reg_moyennes['PCT_MEAN'].max():.1f}%
    • Taux min: {reg_moyennes['PCT_MEAN'].min():.1f}%
    • Écart moyen: {reg_moyennes['PCT_STD'].mean():.1f}%
    """
    fig.text(0.02, 0.02, stats_text, fontsize=8,
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    # Légende
    fig.text(0.5, 0.02,
             f'Source: Analyse Spatiale 2018 | {len(reg_moyennes)} régions analysées | Projection: WGS84',
             ha='center', fontsize=9, style='italic')
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(DOSSIER_SORTIE / '05_carte_regions.png', dpi=300, bbox_inches='tight')
    print(f"✓ 05_carte_regions.png")
    plt.close()


def carte_categories(df_valides, gdf_dept):
    """Carte catégories d'urbanisation avec échelle"""
    # Catégories
    df_valides['NOM_NORM'] = df_valides['NOM_DEPT'].apply(normaliser_noms)
    bins = [0, 1, 5, 10, 20, 100]
    labels = ['Très faible (<1%)', 'Faible (1-5%)', 'Modérée (5-10%)', 
              'Élevée (10-20%)', 'Très élevée (>20%)']
    df_valides['CATEGORIE'] = pd.cut(df_valides['PCT_BATI'], bins=bins, labels=labels)
    
    # Fusion
    gdf_dept['NOM_NORM'] = gdf_dept['NAME_2'].apply(normaliser_noms)
    gdf = gdf_dept.merge(df_valides[['NOM_NORM', 'CATEGORIE', 'NOM_REGION', 'PCT_BATI']], 
                         on='NOM_NORM', how='left')
    
    # Carte
    fig, ax = plt.subplots(figsize=(16, 14))
    fig.suptitle('CLASSIFICATION DE L\'URBANISATION - SÉNÉGAL 2018\nCatégories de Taux d\'Urbanisation par Département',
                 fontsize=18, fontweight='bold', y=0.98)
    
    # Palette catégorielle
    colors = {
        'Très faible (<1%)': '#2ecc71',      # Vert
        'Faible (1-5%)': '#f39c12',         # Orange clair
        'Modérée (5-10%)': '#e67e22',       # Orange
        'Élevée (10-20%)': '#e74c3c',       # Rouge clair
        'Très élevée (>20%)': '#c0392b'     # Rouge foncé
    }
    
    # Tracer chaque catégorie
    for cat in labels:
        gdf_cat = gdf[gdf['CATEGORIE'] == cat]
        if len(gdf_cat) > 0:
            gdf_cat.plot(ax=ax, color=colors[cat], edgecolor='black', linewidth=0.8,
                        label=f'{cat} ({len(gdf_cat)} depts)')
    
    # Départements sans données
    gdf_sans_donnees = gdf[gdf['CATEGORIE'].isna()]
    if len(gdf_sans_donnees) > 0:
        gdf_sans_donnees.plot(ax=ax, color='lightgrey', edgecolor='black',
                              linewidth=0.8, hatch='//', label=f'Pas de données ({len(gdf_sans_donnees)})')
    
    # Ajouter noms des principales villes/régions
    capitales = {
        'DAKAR': (-17.4467, 14.7645),
        'THIÈS': (-16.935, 14.79),
        'SAINT-LOUIS': (-16.4896, 16.022),
        'KAOLACK': (-16.0758, 14.165),
        'ZIGUINCHOR': (-16.2819, 12.564)
    }
    
    for ville, (lon, lat) in capitales.items():
        ax.plot(lon, lat, 'v', color='darkblue', markersize=10)
        ax.annotate(ville, xy=(lon, lat), xytext=(5, 5),
                   textcoords="offset points", fontsize=10, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.9))
    
    ax.set_title('CATÉGORISATION DU DEGRÉ D\'URBANISATION', fontsize=14, fontweight='bold', pad=15)
    ax.axis('off')
    
    # Légende améliorée
    ax.legend(title='NIVEAU D\'URBANISATION', loc='upper left', fontsize=10,
              title_fontsize=11, framealpha=0.9, edgecolor='black')
    
    # Échelle manuelle
    bounds = gdf.total_bounds
    scale_x = bounds[0] + (bounds[2] - bounds[0]) * 0.05
    scale_y = bounds[1] + (bounds[3] - bounds[1]) * 0.05
    scale_deg = 100 / 111
    
    ax.plot([scale_x, scale_x + scale_deg], [scale_y, scale_y], 
            color='black', linewidth=3)
    ax.plot([scale_x, scale_x], [scale_y-0.05, scale_y+0.05], 
            color='black', linewidth=2)
    ax.plot([scale_x + scale_deg, scale_x + scale_deg], [scale_y-0.05, scale_y+0.05], 
            color='black', linewidth=2)
    ax.text(scale_x + scale_deg/2, scale_y - 0.1, '100 km', 
            ha='center', va='top', fontsize=9, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Ajouter flèche nord
    north_x = bounds[0] + (bounds[2] - bounds[0]) * 0.95
    north_y = bounds[1] + (bounds[3] - bounds[1]) * 0.95
    ax.text(north_x, north_y, 'N', fontsize=14, fontweight='bold',
            va='center', ha='center', 
            bbox=dict(boxstyle='circle', facecolor='white', edgecolor='black'))
    
    # Statistiques des catégories
    cat_stats = df_valides['CATEGORIE'].value_counts()
    stats_text = "RÉPARTITION PAR CATÉGORIE:\n"
    total = len(df_valides)
    for cat in labels:
        count = cat_stats.get(cat, 0)
        stats_text += f"• {cat.split('(')[0].strip()}: {count} depts ({count/total*100:.1f}%)\n"
    
    # Ajouter top 5 départements par catégorie
    stats_text += "\nTOP 5 PAR CATÉGORIE:\n"
    for cat in ['Très élevée (>20%)', 'Élevée (10-20%)']:
        cat_depts = df_valides[df_valides['CATEGORIE'] == cat]
        if len(cat_depts) > 0:
            top = cat_depts.nlargest(3, 'PCT_BATI')
            stats_text += f"{cat.split('(')[0]}:\n"
            for _, dept in top.iterrows():
                stats_text += f"  - {dept['NOM_DEPT']}: {dept['PCT_BATI']:.1f}%\n"
    
    fig.text(0.02, 0.15, stats_text, fontsize=9,
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9),
             verticalalignment='top')
    
    # Légende de la carte
    fig.text(0.5, 0.02,
             'Source: Analyse Spatiale 2018 | ▲ Capitales régionales | Classification: PCT_BATI',
             ha='center', fontsize=9, style='italic')
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(DOSSIER_SORTIE / '06_carte_categories.png', dpi=300, bbox_inches='tight')
    print(f"✓ 06_carte_categories.png")
    plt.close()
    
# ============================================
# MAIN
# ============================================

def main():
    print("\n" + "="*70)
    print("VISUALISATION URBANISATION SÉNÉGAL 2018")
    print("="*70)
    
    # Chargement
    df, df, gdf_dept, gdf_region = charger_donnees()
    
    # Si chargement échoué, arrêter
    if df is None:
        print("\n❌ Impossible de continuer. Veuillez corriger les chemins.")
        print(f"   Vérifiez votre structure de dossiers:")
        print(f"   {REPERTOIRE_BASE / 'data'}")
        return
    
    # Stats
    stats_descriptives(df)
    
    # Graphiques
    print("\n" + "="*70)
    print("GÉNÉRATION DES VISUALISATIONS")
    print("="*70 + "\n")
    
    graphique_distribution(df)
    graphique_top_departements(df)
    stats_reg = graphique_regions(df)
    
    # Cartes
    carte_departements(df, gdf_dept)
    carte_regions(df, gdf_region)
    carte_categories(df, gdf_dept)
    
    # Résumé
    print("\n" + "="*70)
    print("✅ VISUALISATIONS TERMINÉES")
    print("="*70)
    print(f"\n📂 Fichiers dans: {DOSSIER_SORTIE}")
    print("\n📋 Fichiers générés:")
    print("  1. 01_distribution.png - Distribution globale")
    print("  2. 02_top_departements.png - Top/Bottom 20")
    print("  3. 03_analyse_regions.png - Analyse régionale")
    print("  4. 04_carte_departements.png - Carte départements")
    print("  5. 05_carte_regions.png - Carte régions")
    print("  6. 06_carte_categories.png - Carte catégories")
    print("\n✨ Analyse terminée avec succès!")

if __name__ == "__main__":
    main()


VISUALISATION URBANISATION SÉNÉGAL 2018
CHARGEMENT DES DONNÉES
📂 Vérification des fichiers...
  - CSV: C:\Users\HP\Documents\ISEP3\Semestre 1_CT\Stat\Stat_Spatiale\TP6\data\senegal_unifie_467_20251222_141659.csv
    Existe: ✓
  - Shapefile régions: C:\Users\HP\Documents\ISEP3\Semestre 1_CT\Stat\Stat_Spatiale\TP6\data\GADM\gadm\gadm41_SEN_shp\gadm41_SEN_1.shp
    Existe: ✓
  - Shapefile départements: C:\Users\HP\Documents\ISEP3\Semestre 1_CT\Stat\Stat_Spatiale\TP6\data\GADM\gadm\gadm41_SEN_shp\gadm41_SEN_2.shp
    Existe: ✓

📊 Chargement CSV...
✓ 45 départements, 45 valides

🗺️  Chargement shapefiles GADM...
✓ 45 départements, 14 régions
✓ Dossier sortie: C:\Users\HP\Documents\ISEP3\Semestre 1_CT\Stat\Stat_Spatiale\TP6\visualisations

📊 STATISTIQUES DESCRIPTIVES

Moyenne: 44.87% | Médiane: 43.40%
Min: 6.29% | Max: 94.52%
Écart-type: 25.89%

GÉNÉRATION DES VISUALISATIONS

✓ 01_distribution.png
✓ 02_top_departements.png
✓ 03_analyse_regions.png

📍 Départements matchés: 45/45 (100.0%)
✓ 0